<a href="https://colab.research.google.com/github/DominiqueLoyer/systemFactChecking/blob/main/Copie_de_Backend_Flask_(app_py).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
from flask import Flask, request, jsonify
import sys
import os
import traceback # To help with debugging errors

# Install flask-cors if not already installed
try:
    import flask_cors
except ImportError:
    print("flask_cors not found, installing...")
    !pip install flask-cors
    import flask_cors

from flask_cors import CORS # To handle Cross-Origin Resource Sharing

# --- Configuration Import ---
# Add the directory containing 'credibility_system.py' to the Python path
# Adjust this path if your file structure is different
# Assumes 'credibility_system.py' is in the same directory as 'app.py'
# If it's elsewhere, provide the correct path.
# Example: sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..', 'path_to_your_module')))
try:
    # Assuming your CredibilityVerificationSystem class is in a file named 'credibility_system.py'
    # in the same directory as this 'app.py' file.
    from credibility_system import CredibilityVerificationSystem
    print("Successfully imported CredibilityVerificationSystem.")
    # --- Instantiate the System ---
    # Load models only once when the server starts
    # This can take time, so the server might be slow to start initially.
    print("Initializing CredibilityVerificationSystem... (This might take a moment)")
    credibility_checker = CredibilityVerificationSystem()
    print("CredibilityVerificationSystem initialized.")

except ImportError:
    print("Error: Could not import CredibilityVerificationSystem.")
    print("Please ensure 'credibility_system.py' exists and is in the correct path.")
    print(f"Current sys.path: {sys.path}")
    # Exit if the core system cannot be imported
    sys.exit("Failed to import credibility system module.")
except Exception as e:
    print(f"Error initializing CredibilityVerificationSystem: {e}")
    traceback.print_exc() # Print detailed error traceback
    sys.exit("Failed to initialize credibility system.")


# --- Flask App Initialization ---
app = Flask(__name__)
# IMPORTANT: Enable CORS to allow requests from your HTML file (frontend)
# For development, allow all origins ('*'). For production, restrict this
# to the actual domain where your frontend is hosted.
CORS(app)
print("Flask app created and CORS enabled.")

# --- API Route Definition ---
@app.route('/api/verify', methods=['POST'])
def verify_endpoint():
    """
    API endpoint to receive input data and return credibility analysis.
    Expects a JSON payload with the key 'input_data'.
    """
    print("\nReceived request on /api/verify")

    # 1. Get data from the request
    if not request.is_json:
        print("Error: Request is not JSON")
        return jsonify({"error": "Request must be JSON"}), 400 # Bad Request

    data = request.get_json()
    input_data = data.get('input_data', None)

    if not input_data or not isinstance(input_data, str) or not input_data.strip():
        print("Error: 'input_data' is missing or invalid")
        return jsonify({"error": "'input_data' field is required and must be a non-empty string"}), 400

    print(f"Processing input: {input_data[:100]}...") # Log received data (truncated)

    # 2. Call the credibility verification system
    try:
        # Use the pre-initialized checker instance
        results = credibility_checker.verify_information(input_data)
        print("Verification successful.")
        # Check if the verification itself returned an error structure
        if isinstance(results, dict) and 'error' in results:
             print(f"Verification returned an error: {results['error']}")
             # Return the specific error from the verification logic
             # Use 400 (Bad Request) or potentially 500 (Internal Server Error)
             # depending on the nature of the error. 400 is often suitable if
             # the input caused the issue (e.g., invalid URL, empty text after processing).
             return jsonify(results), 400

        # 3. Return the results as JSON
        print(f"Returning report with score: {results.get('scoreCredibilite', 'N/A')}")
        return jsonify(results), 200 # OK

    except Exception as e:
        # Catch any unexpected errors during verification
        print(f"Error during verification process: {e}")
        traceback.print_exc() # Log the full error traceback for debugging
        # Return a generic server error message
        return jsonify({"error": "An internal server error occurred during analysis."}), 500 # Internal Server Error


# --- Run the App (for development) ---
if __name__ == '__main__':
    # Runs the Flask development server.
    # host='0.0.0.0' makes it accessible on your network (useful for testing from other devices)
    # debug=True provides automatic reloading and more detailed error pages (DO NOT use in production)
    print("Starting Flask development server...")
    app.run(host='0.0.0.0', port=5000, debug=True)
    # Note: When debug=True, Flask might initialize the app twice,
    # so you might see the "Initializing..." messages twice. This is normal.

ModuleNotFoundError: No module named 'flask_cors'